In [51]:
import os
import numpy as np
import librosa
import skfuzzy as fuzz
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [ ]:

# Function to extract MFCC features
def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc.T, axis=0)  # Take the mean across time
    return mfcc_mean

# Function to save extracted features to file
def extract_and_save_features(wav_dir, seg_label_path, output_dir, max_files=None):
    files = os.listdir(wav_dir)
    
    # Load segment labels for the dataset with allow_pickle=True
    segment_labels = np.load(seg_label_path, allow_pickle=True)
    
    os.makedirs(output_dir, exist_ok=True)

    for i, file_name in enumerate(files[:max_files]):
        file_path = os.path.join(wav_dir, file_name)
        mfcc = extract_mfcc(file_path)

        # Save MFCC features and corresponding segment labels
        feature_save_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_mfcc.npy")
        label_save_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_segment_label.npy")
        np.save(feature_save_path, mfcc)
        np.save(label_save_path, segment_labels[i])

# Function to load utterance level labels
def load_utterance_labels(label_path):
    with open(label_path, 'r') as file:
        labels = file.readlines()
    return np.array([int(label.strip()) for label in labels])  # Convert to numpy array

# Function to load saved MFCC features and segment labels
def load_dataset(feature_dir, max_files=None):
    feature_files = [f for f in os.listdir(feature_dir) if f.endswith('_mfcc.npy')]
    features = []
    labels = []
    
    for i, feature_file in enumerate(feature_files[:max_files]):
        feature_path = os.path.join(feature_dir, feature_file)
        label_path = feature_path.replace('_mfcc.npy', '_segment_label.npy')
        mfcc = np.load(feature_path)
        label = np.load(label_path)
        features.append(mfcc)
        labels.append(label)
    
    return np.array(features), np.array(labels)

# Fully Fake Detection using RRL (Decision Tree)
def rrl_fully_fake_detection(X_train, y_train, X_test):
    clf = DecisionTreeClassifier(max_depth=5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return clf, y_pred

# Fuzzy Logic for Partial Fake Detection and Localization
def fuzzy_partial_localization(mfcc_features):
    x_mfcc = np.arange(0, 1.01, 0.01)
    real_mf = fuzz.trimf(x_mfcc, [0, 0.4, 0.7])
    fake_mf = fuzz.trimf(x_mfcc, [0.3, 0.6, 1])

    mfcc_norm = (mfcc_features - np.min(mfcc_features)) / (np.max(mfcc_features) - np.min(mfcc_features))
    realness = fuzz.interp_membership(x_mfcc, real_mf, mfcc_norm)
    fakeness = fuzz.interp_membership(x_mfcc, fake_mf, mfcc_norm)

    fake_threshold = 0.6
    partial_fake_indices = np.where(fakeness > fake_threshold)[0]

    if len(partial_fake_indices) > 0:
        print("Partial fake detected at indices:", partial_fake_indices)
        return True, partial_fake_indices
    else:
        print("No partial fake detected")
        return False, []

# Main process to load data, train model, and apply hybrid system
if __name__ == "__main__":
    base_output_path = "F:\\Awais_data\\Datasets\\PartialSpoof\\Features\\Frame_level_features"

    train_output_path = os.path.join(base_output_path, "train")
    dev_output_path = os.path.join(base_output_path, "dev")
    eval_output_path = os.path.join(base_output_path, "eval")

    os.makedirs(train_output_path, exist_ok=True)
    os.makedirs(dev_output_path, exist_ok=True)
    os.makedirs(eval_output_path, exist_ok=True)

    # Extract and save MFCC features using segment level labels
    extract_and_save_features("F:\\Awais_data\\Datasets\\PartialSpoof\\Train\\con_wav\\", 
                              "F:\\Awais_data\\Datasets\\PartialSpoof\\database_segment_labels\\database\\segment_labels\\train_seglab_0.16.npy", 
                              train_output_path, max_files=2000)

    extract_and_save_features("F:\\Awais_data\\Datasets\\PartialSpoof\\dev\\con_wav\\", 
                              "F:\\Awais_data\\Datasets\\PartialSpoof\\database_segment_labels\\database\\segment_labels\\dev_seglab_0.16.npy", 
                              dev_output_path, max_files=2000)

    extract_and_save_features("F:\\Awais_data\\Datasets\\PartialSpoof\\eval\\con_wav\\", 
                              "F:\\Awais_data\\Datasets\\PartialSpoof\\database_segment_labels\\database\\segment_labels\\eval_seglab_0.16.npy", 
                              eval_output_path, max_files=1000)

    # Load utterance level labels
    train_utterance_labels = load_utterance_labels("F:\\Awais_data\\Datasets\\PartialSpoof\\protocols\\PartialSpoof_LA_cm_protocols\\PartialSpoof.LA.cm.train.trl.txt")
    dev_utterance_labels = load_utterance_labels("F:\\Awais_data\\Datasets\\PartialSpoof\\protocols\\PartialSpoof_LA_cm_protocols\\PartialSpoof.LA.cm.dev.trl.txt")

    # Load extracted dataset
    print("Loading dataset...")
    X_train, y_train = load_dataset(train_output_path, max_files=2000)  # Adjust max_files as needed
    X_dev, y_dev = load_dataset(dev_output_path, max_files=1000)

    # Fully Fake Detection (RRL) using utterance level labels
    print("Training RRL model for fully fake detection...")
    clf, y_pred_rrl = rrl_fully_fake_detection(X_train, train_utterance_labels[:len(y_train)], X_dev)
    accuracy = accuracy_score(dev_utterance_labels[:len(y_dev)], y_pred_rrl)
    print(f"RRL Fully Fake Detection Accuracy: {accuracy * 100:.2f}%")

    # Fuzzy Logic for Partial Fake Detection on a sample from dev set
    print("Applying fuzzy logic for partial fake detection and localization...")
    sample_mfcc = X_dev[0]  # Example of taking one MFCC sample from dev set
    is_partial_fake, partial_fake_indices = fuzzy_partial_localization(sample_mfcc)

    # Hybrid system conclusion
    if y_pred_rrl[0] == 1:  # Fully fake detection
        print("The entire audio is classified as fully fake.")
    elif is_partial_fake:
        print(f"Partial fake detected in the following regions: {partial_fake_indices}")
    else:
        print("The audio is classified as real.")

    # Save the trained RRL model for future use
    joblib.dump(clf, os.path.join(base_output_path, "rrl_fully_fake_detection_model.joblib"))


In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import librosa
from tqdm import tqdm  # Import tqdm for progress bars

# Load your MFCC features dynamically
def load_audio_features_and_labels(audio_dir, label_file):
    mfcc_features = []
    labels = []

    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()

    # Iterate through audio files with a progress bar
    for audio_file in tqdm(os.listdir(audio_dir), desc="Loading audio files"):
        if audio_file.endswith('.wav'):
            file_id = audio_file[:-4]  # Removing '.wav' to get file ID
            audio_path = os.path.join(audio_dir, audio_file)

            # Load the audio and extract MFCC features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc = mfcc.T  # Transpose to shape (num_segments, num_coefficients)

            # Get the corresponding label
            label = segment_labels[file_id]

            # Check if the label is a string, and convert to numeric if needed
            if isinstance(label, str):
                label = float(label)  # or int(label) depending on your labels

            mfcc_features.append(mfcc)
            labels.append(label)

    return mfcc_features, labels


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Dataset class
class AudioDataset(Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        mfcc = torch.tensor(self.audio_features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return mfcc, label

# Training function
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    # Progress bar for training
    for inputs, targets in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    print(f"Training Loss: {average_loss:.4f}")

# Evaluation function
def evaluate(model, dev_loader):
    model.eval()
    dev_preds = []
    dev_labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dev_loader, desc="Evaluating"):
            outputs = model(inputs)
            dev_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            dev_labels.extend(targets.cpu().numpy())

    return dev_preds, dev_labels

def pad_collate_fn(batch, padding_value=-1, max_length=None):
    features, labels = zip(*batch)
    
    # Find the maximum length in this batch, or use a fixed max length if provided
    max_len = max([feat.shape[0] for feat in features]) if max_length is None else max_length
    
    # Pad or truncate the features and labels to the maximum length
    padded_features = []
    padded_labels = []
    
    for feat, label in zip(features, labels):
        # Pad features
        if feat.shape[0] < max_len:
            # Pad the feature to match max_len
            padded_feat = torch.nn.functional.pad(feat, (0, 0, 0, max_len - feat.shape[0]))
            # Pad the labels with the padding_value
            padded_label = torch.nn.functional.pad(label, (0, max_len - label.shape[0]), value=padding_value)
        else:
            # Truncate features and labels to the max length
            padded_feat = feat[:max_len, :]
            padded_label = label[:max_len]
        
        padded_features.append(padded_feat)
        padded_labels.append(padded_label)
    
    # Stack the padded features and labels into batches
    batch_features = torch.stack(padded_features)
    batch_labels = torch.stack(padded_labels)
    
    return batch_features, batch_labels

# Main function
def main():
    audio_dir_train = "F:/Awais_data/Datasets/PartialSpoof/train/con_wav"
    audio_dir_dev = "F:/Awais_data/Datasets/PartialSpoof/dev/con_wav"
    audio_dir_eval = "F:/Awais_data/Datasets/PartialSpoof/eval/con_wav"
    
    label_file_train = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy"
    label_file_dev = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/dev_seglab_fixed.npy"
    label_file_eval = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_fixed.npy"

    # Load data
    train_mfcc, train_labels = load_audio_features_and_labels(audio_dir_train, label_file_train)
    dev_mfcc, dev_labels = load_audio_features_and_labels(audio_dir_dev, label_file_dev)
    eval_mfcc, eval_labels = load_audio_features_and_labels(audio_dir_eval, label_file_eval)

    # Flattening the features and ensuring consistency
    train_dataset = AudioDataset(train_mfcc, train_labels)
    dev_dataset = AudioDataset(dev_mfcc, dev_labels)
    eval_dataset = AudioDataset(eval_mfcc, eval_labels)

    # Example DataLoader setup with padding/truncating
    train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: pad_collate_fn(x, padding_value=-1, max_length=300))
    dev_loader = torch.utils.data.DataLoader(
    dev_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: pad_collate_fn(x, padding_value=-1, max_length=300))

    # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    # dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

    # Model parameters
    input_size = 13 * 100  # Update this based on your MFCC output
    hidden_size = 64
    output_size = 1  # Binary classification

    model = MLP(input_size, hidden_size, output_size)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_loader, criterion, optimizer)
        dev_preds, dev_labels = evaluate(model, dev_loader)

if __name__ == "__main__":
    main()


Loading audio files: 100%|██████████████████████████████████████████████████████| 71237/71237 [09:00<00:00, 131.89it/s]


Epoch 1/10


Training:   0%|                                                                                | 0/794 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x3900 and 1300x64)

In [47]:
print("Feature shape:", inputs.shape)


NameError: name 'inputs' is not defined

In [44]:
import numpy as np

def check_label_types(label_file):
    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()
    
    # Check each label
    for file_id, label in segment_labels.items():
        # Check the type of the label
        if not isinstance(label, (int, float)):  # Check if it's not int or float
            print(f"Warning: Label for file {file_id} is of type {type(label)} and has value '{label}'")
        else:
            print(f"Label for file {file_id} is valid with type {type(label)} and value '{label}'")

# Replace 'your_label_file.npy' with the actual path to your .npy label file
label_file = 'F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy'
check_label_types(label_file)


 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1.]'
 1. 1. 1.]'
 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1.]'
 1. 1. 1. 1. 1.]'
 1.]'
 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1.]'
 1.]'
 1. 1.]'
 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1.]'
 1.]'
 1. 1. 1. 1.]'
 1. 1. 1.]'
 1.]'
 1.]'
 1. 1. 1.]'
 1. 1. 1. 1.]'
 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1.]'
 1.]'
 1. 1. 1.]'
 1. 1.]'
 1. 1.]'
 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1.]'
 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1.]'
 1. 1. 1.]'
 1. 1.]'
 1. 1.]'
 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1.]'
 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1. 1.]'
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]'
 1. 1.]'
 1. 1. 1.]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [43]:
import numpy as np

# Load the labels with temporal resolution
train_seglab = np.load("F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_0.16.npy", allow_pickle=True).item()

# Function to convert string labels to numeric
def convert_labels(label_dict):
    fixed_labels = {}
    for file_id, labels in label_dict.items():
        # Convert the array of strings ('0', '1') to integers or floats
        if isinstance(labels, np.ndarray) and labels.dtype.type is np.str_:
            fixed_labels[file_id] = labels.astype(float)  # or .astype(int) for integer labels
        else:
            fixed_labels[file_id] = labels  # If labels are already numeric
    return fixed_labels

# Convert all labels to numeric types (float or int)
train_seglab_fixed = convert_labels(train_seglab)

# Save the fixed labels to the specified location
np.save('F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_fixed.npy', train_seglab_fixed)

# Example: Access and print the fixed labels
print("Fixed labels saved at the specified location.")



Fixed labels saved at the specified location.


In [40]:
pwd

'C:\\Notebooks'

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import librosa
from tqdm import tqdm  # Import tqdm for progress bars

# Define a fixed length for MFCC features
FIXED_MFCC_LENGTH = 20  # Adjust this as needed

# Load your MFCC features dynamically
def load_audio_features_and_labels(audio_dir, label_file, limit=None):
    mfcc_features = []
    labels = []

    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()

    # Iterate through audio files with a progress bar
    for audio_file in tqdm(os.listdir(audio_dir), desc="Loading audio files"):
        if audio_file.endswith('.wav'):
            file_id = audio_file[:-4]  # Removing '.wav' to get file ID
            audio_path = os.path.join(audio_dir, audio_file)

            # Load the audio and extract MFCC features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc = mfcc.T  # Transpose to shape (num_segments, num_coefficients)

            # Get the corresponding label
            label = segment_labels[file_id]

            # Check if the label is a string, and convert to numeric if needed
            if isinstance(label, str):
                label = float(label)  # or int(label) depending on your labels
            
            # Make sure label is an array for consistency
            if isinstance(label, (int, float)):
                label = np.array([label])  # Convert to array

            # Pad or truncate the MFCC features
            if mfcc.shape[0] < FIXED_MFCC_LENGTH:
                # Pad the MFCC array
                padded_mfcc = np.pad(mfcc, ((0, FIXED_MFCC_LENGTH - mfcc.shape[0]), (0, 0)), mode='constant')
            else:
                # Truncate the MFCC array
                padded_mfcc = mfcc[:FIXED_MFCC_LENGTH]

            mfcc_features.append(padded_mfcc)
            labels.append(label)

            if limit is not None and len(mfcc_features) >= limit:
                break  # Stop loading if we reach the limit

    return mfcc_features, labels

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Change input_size based on your flattened input shape
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Ensure it's a flat vector
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Dataset class
class AudioDataset(Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        mfcc = torch.tensor(self.audio_features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return mfcc, label

def train(model, train_loader, criterion, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss, ignoring the padded labels (-1)
        mask = labels != -1
        loss = masked_bce_loss(outputs, labels, mask)

        loss.backward()
        optimizer.step()


# Evaluation function
def evaluate(model, dev_loader):
    model.eval()
    dev_preds = []
    dev_labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dev_loader, desc="Evaluating"):
            outputs = model(inputs)
            dev_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            dev_labels.extend(targets.cpu().numpy())

    return dev_preds, dev_labels

def pad_labels(label, max_len):
    # Pad labels with a special value (e.g., -1) to match the maximum length
    padded_label = F.pad(label, (0, max_len - len(label)), value=-1)
    return padded_label

# Example in your training loop:
for feat, label in dataset:
    # Assume features are already padded to max_len (e.g., 30 frames)
    max_len = feat.shape[0]  # Number of frames in the padded feature tensor
    padded_label = pad_labels(label, max_len)
    
    # Add padded_label to the batch
    train_batch.append((feat, padded_label))

def pad_collate_fn(batch, padding_value=0):
    padded_features = []
    padded_labels = []
    
    max_len_feat = max([feat.shape[0] for feat, label in batch])  # Maximum feature length
    max_len_label = max([len(label) for feat, label in batch])    # Maximum label length
    
    for feat, label in batch:
        # Pad features
        padded_feat = torch.tensor(feat, dtype=torch.float32)
        padded_feat = F.pad(padded_feat, (0, 0, 0, max_len_feat - feat.shape[0]), value=padding_value)
        
        # Pad labels
        padded_label = pad_labels(torch.tensor(label, dtype=torch.float32), max_len_label)

        print(f"Feature shape: {padded_feat.shape}, Label shape: {padded_label.shape}")  # Debugging line
        
        padded_features.append(padded_feat)
        padded_labels.append(padded_label)

    # Stack sequences into tensors
    padded_features = torch.stack(padded_features)
    padded_labels = torch.stack(padded_labels)

    return padded_features, padded_labels

 
def masked_bce_loss(outputs, targets, mask):
    # Apply the mask to both outputs and targets
    outputs_masked = outputs[mask]
    targets_masked = targets[mask]
    return F.binary_cross_entropy_with_logits(outputs_masked, targets_masked)
    
def main():
    audio_dir_train = "F:/Awais_data/Datasets/PartialSpoof/train/con_wav"
    audio_dir_dev = "F:/Awais_data/Datasets/PartialSpoof/dev/con_wav"
    audio_dir_eval = "F:/Awais_data/Datasets/PartialSpoof/eval/con_wav"
    
    label_file_train = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy"
    label_file_dev = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/dev_seglab_fixed.npy"
    label_file_eval = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_fixed.npy"

    # Load data (limit to 1000 samples)
    train_mfcc, train_labels = load_audio_features_and_labels(audio_dir_train, label_file_train, limit=1000)
    dev_mfcc, dev_labels = load_audio_features_and_labels(audio_dir_dev, label_file_dev, limit=1000)
    eval_mfcc, eval_labels = load_audio_features_and_labels(audio_dir_eval, label_file_eval, limit=1000)

    # Print dimensions for confirmation (only first 10 shapes)
    print(f"Train MFCC shape (first 10): {[mfcc.shape for mfcc in train_mfcc[:10]]}")
    print(f"Train labels (first 10): {train_labels[:10]}")

    print(f"Dev MFCC shape (first 10): {[mfcc.shape for mfcc in dev_mfcc[:10]]}")
    print(f"Dev labels (first 10): {dev_labels[:10]}")

    print(f"Eval MFCC shape (first 10): {[mfcc.shape for mfcc in eval_mfcc[:10]]}")
    print(f"Eval labels (first 10): {eval_labels[:10]}")

    # Flattening the features and ensuring consistency
    train_dataset = AudioDataset(train_mfcc, train_labels)
    dev_dataset = AudioDataset(dev_mfcc, dev_labels)
    eval_dataset = AudioDataset(eval_mfcc, eval_labels)

    # Example DataLoader setup with padding/truncating
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate_fn)
    dev_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate_fn)

    # Model parameters
    input_size = FIXED_MFCC_LENGTH * 13  # Each MFCC feature vector has 13 coefficients
    hidden_size = 64
    output_size = 1  # Binary classification

    model = MLP(input_size, hidden_size, output_size)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_loader, criterion, optimizer)
        dev_preds, dev_labels = evaluate(model, dev_loader)

if __name__ == "__main__":
    main()


NameError: name 'dataset' is not defined

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import librosa
from tqdm import tqdm  # Import tqdm for progress bars
import torch.nn.functional as F

# Define a fixed length for MFCC features
FIXED_MFCC_LENGTH = 20  # Adjust this as needed

# Load your MFCC features dynamically
def load_audio_features_and_labels(audio_dir, label_file, limit=None):
    mfcc_features = []
    labels = []

    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()

    # Iterate through audio files with a progress bar
    for audio_file in tqdm(os.listdir(audio_dir), desc="Loading audio files"):
        if audio_file.endswith('.wav'):
            file_id = audio_file[:-4]  # Removing '.wav' to get file ID
            audio_path = os.path.join(audio_dir, audio_file)

            # Load the audio and extract MFCC features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc = mfcc.T  # Transpose to shape (num_segments, num_coefficients)

            # Get the corresponding label
            label = segment_labels[file_id]

            # Check if the label is a string, and convert to numeric if needed
            if isinstance(label, str):
                label = float(label)  # or int(label) depending on your labels
            
            # Make sure label is an array for consistency
            if isinstance(label, (int, float)):
                label = np.array([label])  # Convert to array

            # Pad or truncate the MFCC features
            if mfcc.shape[0] < FIXED_MFCC_LENGTH:
                # Pad the MFCC array
                padded_mfcc = np.pad(mfcc, ((0, FIXED_MFCC_LENGTH - mfcc.shape[0]), (0, 0)), mode='constant')
            else:
                # Truncate the MFCC array
                padded_mfcc = mfcc[:FIXED_MFCC_LENGTH]

            mfcc_features.append(padded_mfcc)
            labels.append(label)

            if limit is not None and len(mfcc_features) >= limit:
                break  # Stop loading if we reach the limit

    return mfcc_features, labels

# Define the MLP class
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Adjust input size based on input shape
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if needed
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Dataset class
class AudioDataset(Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        mfcc = torch.tensor(self.audio_features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return mfcc, label

def train(model, train_loader, criterion, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss, ignoring the padded labels (-1)
        mask = labels != -1
        loss = masked_bce_loss(outputs, labels, mask)

        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, dev_loader):
    model.eval()
    dev_preds = []
    dev_labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dev_loader, desc="Evaluating"):
            outputs = model(inputs)
            dev_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            dev_labels.extend(targets.cpu().numpy())

    return dev_preds, dev_labels

def pad_labels(label, max_len):
    # Pad labels with a special value (e.g., -1) to match the maximum length
    padded_label = F.pad(label, (0, max_len - len(label)), value=-1)
    return padded_label

def pad_collate_fn(batch, padding_value=0):
    padded_features = []
    padded_labels = []
    
    max_len_feat = max([feat.shape[0] for feat, label in batch])  # Maximum feature length
    max_len_label = max([len(label) for feat, label in batch])    # Maximum label length
    
    for feat, label in batch:
        # Pad features
        padded_feat = torch.tensor(feat, dtype=torch.float32)
        padded_feat = F.pad(padded_feat, (0, 0, 0, max_len_feat - feat.shape[0]), value=padding_value)
        
        # Pad labels
        padded_label = pad_labels(torch.tensor(label, dtype=torch.float32), max_len_label)

        print(f"Feature shape: {padded_feat.shape}, Label shape: {padded_label.shape}")  # Debugging line
        
        padded_features.append(padded_feat)
        padded_labels.append(padded_label)

    # Stack sequences into tensors
    padded_features = torch.stack(padded_features)
    padded_labels = torch.stack(padded_labels)

    return padded_features, padded_labels

def masked_bce_loss(outputs, targets, mask):
    # Apply the mask to both outputs and targets
    outputs_masked = outputs[mask]
    targets_masked = targets[mask]
    return F.binary_cross_entropy_with_logits(outputs_masked, targets_masked)
    
def main():
    audio_dir_train = "F:/Awais_data/Datasets/PartialSpoof/train/con_wav"
    audio_dir_dev = "F:/Awais_data/Datasets/PartialSpoof/dev/con_wav"
    audio_dir_eval = "F:/Awais_data/Datasets/PartialSpoof/eval/con_wav"
    
    label_file_train = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy"
    label_file_dev = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/dev_seglab_fixed.npy"
    label_file_eval = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_fixed.npy"

    # Load data (limit to 1000 samples)
    train_mfcc, train_labels = load_audio_features_and_labels(audio_dir_train, label_file_train, limit=1000)
    dev_mfcc, dev_labels = load_audio_features_and_labels(audio_dir_dev, label_file_dev, limit=1000)
    eval_mfcc, eval_labels = load_audio_features_and_labels(audio_dir_eval, label_file_eval, limit=1000)

    # Print dimensions for confirmation (only first 10 shapes)
    print(f"Train MFCC shape (first 10): {[mfcc.shape for mfcc in train_mfcc[:10]]}")
    print(f"Train labels (first 10): {train_labels[:10]}")

    print(f"Dev MFCC shape (first 10): {[mfcc.shape for mfcc in dev_mfcc[:10]]}")
    print(f"Dev labels (first 10): {dev_labels[:10]}")

    print(f"Eval MFCC shape (first 10): {[mfcc.shape for mfcc in eval_mfcc[:10]]}")
    print(f"Eval labels (first 10): {eval_labels[:10]}")

    # Flattening the features and ensuring consistency
    train_dataset = AudioDataset(train_mfcc, train_labels)
    dev_dataset = AudioDataset(dev_mfcc, dev_labels)
    eval_dataset = AudioDataset(eval_mfcc, eval_labels)

    # Example DataLoader setup with padding/truncating
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate_fn)
    dev_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate_fn)

    # Model parameters
    input_size = FIXED_MFCC_LENGTH * 13  # Each MFCC feature vector has 13 coefficients
    hidden_size = 64
    output_size = 1  # Binary classification

    model = MLP(input_size, hidden_size, output_size)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_loader, criterion, optimizer)
        dev_preds, dev_labels = evaluate(model, dev_loader)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy'

In [45]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import librosa
from tqdm import tqdm  # Import tqdm for progress bars
import torch.nn.functional as F

# Define a fixed length for MFCC features
FIXED_MFCC_LENGTH = 20  # Adjust this as needed

# Load your MFCC features dynamically
def load_audio_features_and_labels(audio_dir, label_file, limit=None):
    mfcc_features = []
    labels = []

    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()

    # Iterate through audio files with a progress bar
    for audio_file in tqdm(os.listdir(audio_dir), desc="Loading audio files"):
        if audio_file.endswith('.wav'):
            file_id = audio_file[:-4]  # Removing '.wav' to get file ID
            audio_path = os.path.join(audio_dir, audio_file)

            # Load the audio and extract MFCC features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc = mfcc.T  # Transpose to shape (num_segments, num_coefficients)

            # Get the corresponding label
            label = segment_labels[file_id]

            # Check if the label is a string, and convert to numeric if needed
            if isinstance(label, str):
                label = float(label)  # or int(label) depending on your labels
            
            # Make sure label is an array for consistency
            if isinstance(label, (int, float)):
                label = np.array([label])  # Convert to array

            # Pad or truncate the MFCC features
            if mfcc.shape[0] < FIXED_MFCC_LENGTH:
                # Pad the MFCC array
                padded_mfcc = np.pad(mfcc, ((0, FIXED_MFCC_LENGTH - mfcc.shape[0]), (0, 0)), mode='constant')
            else:
                # Truncate the MFCC array
                padded_mfcc = mfcc[:FIXED_MFCC_LENGTH]

            mfcc_features.append(padded_mfcc)
            labels.append(label)

            if limit is not None and len(mfcc_features) >= limit:
                break  # Stop loading if we reach the limit

    return mfcc_features, labels

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, max_seq_len):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, max_seq_len)  # Output matches max_seq_len

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Output shape: [batch_size, max_seq_len]
        return x

# Dataset class
class AudioDataset(Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        mfcc = torch.tensor(self.audio_features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return mfcc, label

def train(model, train_loader, criterion, optimizer):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)  # Outputs will have shape [batch_size, max_len_labels]

        # Compute the mask (should be the same shape as labels)
        mask = (labels != -1)  # Mask will have shape [batch_size, max_len_labels]
        # print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")
        # print(f"Mask shape: {mask.shape}")

        # Calculate the loss using the masked BCE function
        loss = masked_bce_loss(outputs, labels, mask)

        # Backward pass
        loss.backward()
        optimizer.step()

def evaluate(model, dev_loader):
    model.eval()
    dev_preds = []
    dev_labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dev_loader, desc="Evaluating"):
            outputs = model(inputs)
            dev_preds.extend(torch.sigmoid(outputs).cpu().numpy())  # Get predictions
            dev_labels.extend(targets.cpu().numpy())  # Ensure this is a NumPy array

    return np.array(dev_preds), np.array(dev_labels)  


def pad_labels(label, max_len):
    # Ensure label is padded correctly to the maximum label length
    padded_label = F.pad(label, (0, max_len - label.size(0)), value=-1)  # Padding with -1 for masked loss
    return padded_label
    
def load_audio_features_and_labels(audio_dir, label_file, limit=None):
    mfcc_features = []
    labels = []

    # Load the segment labels
    segment_labels = np.load(label_file, allow_pickle=True).item()

    # Iterate through audio files with a progress bar
    for audio_file in tqdm(os.listdir(audio_dir), desc="Loading audio files"):
        if audio_file.endswith('.wav'):
            file_id = audio_file[:-4]  # Removing '.wav' to get file ID
            audio_path = os.path.join(audio_dir, audio_file)

            # Load the audio and extract MFCC features
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            mfcc = mfcc.T  # Transpose to shape (num_segments, num_coefficients)

            # Get the corresponding label
            label = segment_labels[file_id]

            # Ensure label is an array
            if isinstance(label, str):
                label = float(label)  # or int(label) depending on your labels
            label = np.array([label]) if isinstance(label, (int, float)) else label

            # Pad or truncate the MFCC features
            padded_mfcc = np.pad(mfcc, ((0, max(0, FIXED_MFCC_LENGTH - mfcc.shape[0])), (0, 0)), mode='constant')[:FIXED_MFCC_LENGTH]

            mfcc_features.append(padded_mfcc)
            labels.append(label)

            if limit is not None and len(mfcc_features) >= limit:
                break  # Stop loading if we reach the limit

    return mfcc_features, labels

def pad_collate_fn(batch, padding_value=0):
    padded_features = []
    padded_labels = []
    
    max_len_feat = FIXED_MFCC_LENGTH  # Force fixed length
    max_len_label = FIXED_MFCC_LENGTH  # Force fixed length

    for feat, label in batch:
        # Pad features to fixed length
        padded_feat = torch.tensor(feat, dtype=torch.float32)
        padded_feat = F.pad(padded_feat, (0, 0, 0, max_len_feat - feat.shape[0]), value=padding_value)
        padded_feat = padded_feat[:max_len_feat]  # Ensure fixed length

        # Pad labels to fixed length
        padded_label = torch.tensor(label, dtype=torch.float32)
        padded_label = F.pad(padded_label, (0, max_len_label - len(label)), value=-1)
        padded_label = padded_label[:max_len_label]  # Ensure fixed length

        padded_features.append(padded_feat)
        padded_labels.append(padded_label)

    # Stack sequences into tensors
    padded_features = torch.stack(padded_features)
    padded_labels = torch.stack(padded_labels)

    return padded_features, padded_labels


def masked_bce_loss(outputs, targets, mask):
    # Ensure the mask shape matches the output and target shapes
    outputs_masked = outputs[mask]
    targets_masked = targets[mask]
    return F.binary_cross_entropy_with_logits(outputs_masked, targets_masked)

def calculate_accuracy(preds, targets):
    preds_binary = (preds > 0.5).astype(int)  # Convert all predictions to binary
    targets_binary = (targets > 0.5).astype(int)  # Convert targets to binary as well

    # Ensure that preds_binary and targets_binary are numpy arrays for comparison
    preds_binary = np.array(preds_binary).flatten()
    targets_binary = np.array(targets_binary).flatten()

    correct = (preds_binary == targets_binary).sum()  # Count the number of correct predictions
    accuracy = correct / len(targets_binary) if len(targets_binary) > 0 else 0  # Prevent division by zero
    return accuracy


def calculate_eer(preds, targets, pos_label):
    # Convert predictions and targets to binary based on the threshold of 0.5
    preds_binary = (preds > 0.5).astype(int)  # Convert predictions to binary
    targets_binary = (targets > 0.5).astype(int)  # Convert targets to binary

    # Ensure that preds_binary and targets_binary are numpy arrays for comparison
    preds_binary = np.array(preds_binary)
    targets_binary = np.array(targets_binary)

    # Select the specified class for EER calculation
    binary_preds = preds_binary[:, pos_label]  # Predictions for the specified class
    binary_targets = targets_binary[:, pos_label]  # Targets for the specified class

    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(binary_targets, binary_preds)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]  # This is the EER value
    return eer


def main():
    # Define paths for audio and label files
    audio_dir_train = "F:/Awais_data/Datasets/PartialSpoof/train/con_wav"
    audio_dir_dev = "F:/Awais_data/Datasets/PartialSpoof/dev/con_wav"
    audio_dir_eval = "F:/Awais_data/Datasets/PartialSpoof/eval/con_wav"
    
    label_file_train = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/train_seglab_fixed.npy"
    label_file_dev = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/dev_seglab_fixed.npy"
    label_file_eval = "F:/Awais_data/Datasets/PartialSpoof/database_segment_labels/database/segment_labels/eval_seglab_fixed.npy"

    # Load data (limit to 1000 samples)
    train_mfcc, train_labels = load_audio_features_and_labels(audio_dir_train, label_file_train, limit=25000)
    dev_mfcc, dev_labels = load_audio_features_and_labels(audio_dir_dev, label_file_dev, limit=24000)
    eval_mfcc, eval_labels = load_audio_features_and_labels(audio_dir_eval, label_file_eval, limit=71000)

    # Flattening the features and ensuring consistency
    train_dataset = AudioDataset(train_mfcc, train_labels)
    dev_dataset = AudioDataset(dev_mfcc, dev_labels)
    eval_dataset = AudioDataset(eval_mfcc, eval_labels)

    # Example DataLoader setup with padding/truncating
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate_fn)
    dev_loader = torch.utils.data.DataLoader(
        dev_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate_fn)

    # Model parameters
    input_size = FIXED_MFCC_LENGTH * 13  # Each MFCC feature vector has 13 coefficients
    hidden_size = 64
    output_size = 20  # Binary classification

    model = MLP(input_size, hidden_size, output_size)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_loader, criterion, optimizer)
        dev_preds, dev_labels = evaluate(model, dev_loader)
        print(f'Accuracy :{calculate_accuracy(dev_preds, dev_labels)}')
        
        # Specify the pos_label for EER calculation
        pos_label = 0  # Change this based on which class you want to evaluate
        eer = calculate_eer(dev_preds, dev_labels, pos_label)
        
        # Print EER formatted to 2 decimal places
        print(f'EER : {eer:.2f}') 

if __name__ == "__main__":
    main()

Loading audio files: 100%|████████████████████████████████████████████████████▊| 70999/71237 [2:40:13<00:32,  7.38it/s]
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_feat = torch.tensor(feat, dtype=torch.float32)
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_label = torch.tensor(label, dtype=torch.float32)


Epoch 1/100


Evaluating: 100%|███████████████████████████████████████████████████████████████████| 750/750 [00:02<00:00, 320.37it/s]
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_feat = torch.tensor(feat, dtype=torch.float32)
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_label = torch.tensor(label, dtype=torch.float32)


Accuracy :0.6238416666666666
EER : 0.00
Epoch 2/100


Evaluating: 100%|███████████████████████████████████████████████████████████████████| 750/750 [00:02<00:00, 319.68it/s]
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_feat = torch.tensor(feat, dtype=torch.float32)
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_label = torch.tensor(label, dtype=torch.float32)


...

Accuracy :0.6209708333333334
EER : 0.00
Epoch 99/100


Evaluating: 100%|███████████████████████████████████████████████████████████████████| 750/750 [00:02<00:00, 297.53it/s]
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_feat = torch.tensor(feat, dtype=torch.float32)
C:\Users\mawais\AppData\Local\Temp\ipykernel_10340\16796884.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_label = torch.tensor(label, dtype=torch.float32)


Accuracy :0.6238416666666666
EER : 0.00
Epoch 100/100


Evaluating: 100%|███████████████████████████████████████████████████████████████████| 750/750 [00:02<00:00, 293.74it/s]


Accuracy :0.6238416666666666
EER : 0.00
